##### Copyright 2020 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2020 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/boundless"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.orgで表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/hub/tutorials/boundless.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colabで実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/hub/tutorials/boundless.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/hub/tutorials/boundless.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
  <td><a href="https://tfhub.dev/s?q=google%2Fboundless"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TFハブモデルを参照してください</a></td>
</table>

# Boundless Colab

Boundless モデル Colab へようこそ！このノートブックでは、画像のモデルを実行し、結果を可視化するまでの手順を説明します。

## 概要

Boundless は画像外挿用のモデルです。このモデルは画像を受け取り、その画像の一部分 ([1/2](https://tfhub.dev/google/boundless/half/1)、[1/4](https://tfhub.dev/google/boundless/quarter/1)、[3/4](https://tfhub.dev/google/boundless/three_quarter/1)) を内部的にマスクし、マスクされた部分を補完します。詳細は [Boundless: 画像拡張のための敵対的生成ネットワーク](https://arxiv.org/pdf/1908.07007.pdf)または TensorFlow Hub のモデルに関するドキュメントをご覧ください。

## インポートとセットアップ

基本のインポートから始めます。

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from io import BytesIO
from PIL import Image as PilImage
import numpy as np
from matplotlib import pyplot as plt
from six.moves.urllib.request import urlopen

## 入力する画像を読み取る

util メソッドを作成して画像を読み込み、モデル用にフォーマットしてみましょう (257x257x3)。このメソッドは歪みを避けるために画像を正方形にトリミングし、ローカルの画像やインターネットからの画像を使用することも可能です。

In [ ]:
  def read_image(filename):
    fd = None
    if(filename.startswith('http')):
      fd = urlopen(filename)
    else:
      fd = tf.io.gfile.GFile(filename, 'rb')

    pil_image = PilImage.open(fd)
    width, height = pil_image.size
    # crop to make the image square
    pil_image = pil_image.crop((0, 0, height, height))
    pil_image = pil_image.resize((257,257),PilImage.ANTIALIAS)
    image_unscaled = np.array(pil_image)
    image_np = np.expand_dims(
        image_unscaled.astype(np.float32) / 255., axis=0)
    return image_np

## 可視化メソッド

また可視化メソッドを作成して、モデルが生成したマスクされたバージョンと「塗りつぶされた」バージョンの両方で、元の画像が並べて表示されるようにします。

In [ ]:
def visualize_output_comparison(img_original, img_masked, img_filled):
  plt.figure(figsize=(24,12))
  plt.subplot(131)
  plt.imshow((np.squeeze(img_original)))
  plt.title("Original", fontsize=24)
  plt.axis('off')
  plt.subplot(132)
  plt.imshow((np.squeeze(img_masked)))
  plt.title("Masked", fontsize=24)
  plt.axis('off')
  plt.subplot(133)
  plt.imshow((np.squeeze(img_filled)))
  plt.title("Generated", fontsize=24)
  plt.axis('off')
  plt.show()

## 画像を読み込む

ここではサンプル画像を読み込みますが、独自の画像を Colab にアップロードしてご自由にお試しください。なお、モデルには人物画像に関する制限があるので注意してください。

In [ ]:
wikimedia = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Nusfjord_road%2C_2010_09.jpg/800px-Nusfjord_road%2C_2010_09.jpg"
# wikimedia = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/Beech_forest_M%C3%A1tra_in_winter.jpg/640px-Beech_forest_M%C3%A1tra_in_winter.jpg"
# wikimedia = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/Marmolada_Sunset.jpg/640px-Marmolada_Sunset.jpg"
# wikimedia = "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Aegina_sunset.jpg/640px-Aegina_sunset.jpg"

input_img = read_image(wikimedia)

## TensorFlow Hub からモデルを選択する

TensorFlow Hub には、Half (1/2)、Quarter (1/4)、Three Quarters (3/4) という 3 つのバージョンの Boundless モデルがあります。以下のセルでは、そのうちの任意の 1 つを選択して独自のイメージで試すことができます。別のバージョンのモデルで試す場合には、試したいバージョンを選択してから以下のセルを実行してください。

In [ ]:
#@title Model Selection { display-mode: "form" }
model_name = 'Boundless Quarter' # @param ['Boundless Half', 'Boundless Quarter', 'Boundless Three Quarters']
model_handle_map = {
    'Boundless Half' : 'https://tfhub.dev/google/boundless/half/1',
    'Boundless Quarter' : 'https://tfhub.dev/google/boundless/quarter/1', 
    'Boundless Three Quarters' : 'https://tfhub.dev/google/boundless/three_quarter/1'
}

model_handle = model_handle_map[model_name]


必要なモデルを選択したら、それを TensorFlow Hub から読み込みましょう。

**注意**: モデルのドキュメントを読むには、ブラウザをモデルのハンドルにポイントします。

In [ ]:
print("Loading model {} ({})".format(model_name, model_handle))
model = hub.load(model_handle)

## 推論を行う

Boundless モデルには 2 つの出力があります。

- マスクされた入力画像
- 外挿が完了したマスクされた画像

これら 2 つの画像を可視化し、比較表示します。

In [ ]:
result = model.signatures['default'](tf.constant(input_img))
generated_image =  result['default']
masked_image = result['masked_image']

visualize_output_comparison(input_img, masked_image, generated_image)